In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
import traceback
from copy import deepcopy
from functools import partial, wraps
from itertools import chain
from importlib import reload
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import schedule
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser

from api.watchdog import specs
from api.watchdog.specs import get_trades, get_targets
from api.watchdog.pricer import Pricer
from api.instruments.future import FutureChain, FutureBasket
from api.instruments.strategies2 import get_base_stop
from api.jobs.risk import daily_var_email
from api.jobs.risk_intraday import var_reduction_email, var_reduction_email2
from api.jobs.macd_watcher import macd_signals_email, make_plotly_fig
from api.jobs.positions import history_plotter_email

from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools_clarion import get_clarion_positions
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_specific_future,
    get_root
)
from gioutils.tools_clarion import get_history_plotter_book

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [29]:
bhs.static()

In [30]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")
clear_output()

In [5]:
super_clarion(clarion)
runner = Runner(clarion)

In [32]:
daily_var_email(book='MM', send_email=True)
clear_output()

In [33]:
daily_var_email(book='MU', send_email='mic')
clear_output()

In [1]:
var_reduction_email2(clarion, bq, use_best_var=True, send_email=True)

NameError: name 'var_reduction_email2' is not defined

In [28]:
m = macd_signals_email(bq=bq, send_email='ext')

In [ ]:
pnl_day = clarion.metric(
    "pnl_day",
    "PL",
    "Any",
    {'Period': 'Day'},
)
pnl_two_day = clarion.metric(
    "pnl_two_day",
    "PL",
    "Any",
    {'Period': 'TwoDay'},
)
pnl_ytd = clarion.metric(
    "pnl_ytd",
    "PL",
    "Any",
    {'Period': 'YearToDate'},
)
pnl_mtd = clarion.metric(
    "pnl_mtd",
    "PL",
    "Any",
    {'Period': 'MonthToDate'},
)

pnl_metrics = [
    pnl_day,
    pnl_mtd,
    pnl_ytd,
]

In [ ]:
[m.label for label in metrics]

In [ ]:
clarion.positions.run(

In [ ]:
p = get_clarion_positions(clarion, date='2023-03-15', metrics=pnl_metrics)

In [ ]:
p.query('Strategy == "MM:EUR FLY"').iloc[:, :15]

In [ ]:
p.query('Strategy == "MM:USD OPEN OPT"').iloc[:, :15]

In [ ]:
p.query('Strategy == "MM:USD OPEN OPT"').groupby('Description')[['Notional', 'pnl_mtd', 'pnl_ytd']].sum()

In [ ]:
p.query('Strategy == "MM:USD OPEN OPT"')

In [ ]:
runner = Runner(clarion)

In [ ]:
clarion.instruments()

In [ ]:
er = FutureBasket(
    [

        FutureChain('er5', notional=1),
        FutureChain('er11', notional=-1)
    ]
)

ff = FutureBasket(
    [

        FutureChain('ff5', notional=1),
        FutureChain('ff9', notional=-1)
    ]
)

In [ ]:
get_base_stop(er.quote)

In [ ]:
get_base_stop(ff.quote)

In [ ]:
var_reduction_email2(clarion, bq, send_email='test')

In [ ]:
var_reduction_email(clarion)

In [ ]:
for key, strat in m[1].items():
    strat.run()

In [ ]:
m[1].keys()

In [ ]:
13.5 - 6.5

In [ ]:
1000 * 25 * 1.5

In [ ]:
m[2].rename({key: strat.instrument.name for key, strat in m[1].items()}, axis=0).set_index('tickers', append=True)

In [ ]:
m[2].style.format('{:.2f}', subset='quote').format('{:%d%b%Y}', subset='date')

In [ ]:
500 * 1.23

In [ ]:
615 * 2

In [ ]:
2000 / 3

In [ ]:
for key in m[0]:
    display(m[0][key]._pnl['pv'].plot(title=key))

In [ ]:
strat = deepcopy(m[0]['sfi (4, 8, 12)'])

In [ ]:
get_generic_future(['ffm3', 'ffx3', 'erm4', 'erm5'], bq=bq)

In [ ]:
(1500 * 4167) / (2000 * 2500 * 1.079)

In [ ]:
m[0].keys()

In [ ]:
strat.use_crossover_rule = True

In [ ]:
strat.run()

In [ ]:
strat._pnl['pv'].plot()

In [ ]:
strat.base_stop_periods = 66
strat.base_stop_mult = 1
strat.base_take_ratio = 2
strat.use_crossover_rule = False

In [ ]:
strat.signal.signal_threshold_buy = - strat.signal._data['macd'][-196:].std() * 0.5
strat.signal.signal_threshold_sell = strat.signal._data['macd'][-196:].std() * 0.5
strat.signal.slope_threshold_buy = 0
strat.signal.slope_threshold_sell = 0

In [ ]:
strat

In [ ]:
strat.run()

In [ ]:
strat._pnl['pv'].plot()

In [ ]:
strat.summary

In [ ]:
strat._summary

In [ ]:
macdstr = bq.bdh('.MACDSTR Index', 'px_last', start_date='20220101').pivot(index='date', columns='security', values='px_last').squeeze()

In [ ]:
macdstr.diff().iloc[-7:].std() * 2

In [ ]:
a.a = 5

In [ ]:
a.b

In [ ]:
raise AttributeError

In [ ]:
m[1]['er4'].instrument.quote

In [ ]:
A(3).reinit(5)

In [ ]:
m[1]['sfi (6, 11)'].instrument.age()

In [ ]:
strat = m[1]['sfi (6, 11)']

In [ ]:
new_s = deepcopy(strat.signal)

In [ ]:
es.age().quote.tail()

In [ ]:
d1 = new_s.get_data(es)
d2 = new_s.get_data(es.age())
display(d1.tail())
display(d2.tail())

In [ ]:
new_s

In [ ]:
strat.signal._data

In [ ]:
d

In [ ]:
m[1]

In [ ]:
pd.Timestamp.now()

In [ ]:
millify(pd.Timestamp.now().floor('d'))

In [ ]:
m[2].style.format(partial(millify, decimal=4, pct=None))

In [ ]:
m[0]['er1']._pnl.loc[m[0]['er1']._summary.iloc[-1:]['entry_date']].reset_index().squeeze()

In [ ]:
from datetime import datetime, timezone

In [ ]:
datetime.now(tz=timezone.utc)

In [ ]:
datetime.utcnow()

In [ ]:
pd.Timestamp.utcnow()

In [ ]:
d = history_plotter_email(clarion, bq, delta_mul=-1, delta_tol=499, aged_bottom=True)

In [ ]:
d[1].to_clipboard(header=None)

In [ ]:
d[1]

In [ ]:
d[1].to_clipboard()